# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [51]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')
from dateutil.relativedelta import relativedelta
import math

In [2]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("../data/wikipedia_train3.csv")
test = pd.read_csv("../data/wikipedia_test3.csv")

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [3]:
train.sample(10)

,Page,date,Visits
2504060,山田涼介_ja.wikipedia.org_all-access_all-agents,2016-07-30,2994.0
803553,Baron_Zemo_en.wikipedia.org_desktop_all-agents,2016-03-09,1462.0
779772,Bundespräsident_(Deutschland)_de.wikipedia.org...,2016-03-06,3015.0
669038,Pretty_Little_Liars_(season_6)_en.wikipedia.or...,2016-02-26,6289.0
631226,Agent_Carter_(TV_series)_en.wikipedia.org_all-...,2016-02-23,12376.0
2246043,Everest_(2015_film)_en.wikipedia.org_desktop_a...,2016-07-09,2388.0
1995808,吉田羊_ja.wikipedia.org_all-access_all-agents,2016-06-17,2765.0
2715621,Chris_Evans_es.wikipedia.org_mobile-web_all-ag...,2016-08-17,1188.0
1273745,Mesopotamia_es.wikipedia.org_all-access_all-ag...,2016-04-17,3707.0
241117,Rooting_(Android_OS)_en.wikipedia.org_all-acce...,2016-01-21,9962.0


In [4]:
test.sample(10)

,Page,date,Visits
641019,Gerald_Ford_en.wikipedia.org_all-access_all-ag...,2016-11-03,10120.0
43043,The_BFG_en.wikipedia.org_mobile-web_all-agents,2016-09-13,3328.0
413426,Schweden_de.wikipedia.org_all-access_all-agents,2016-10-14,2203.0
418901,Supergirl_(TV_series)_en.wikipedia.org_all-acc...,2016-10-15,20397.0
163000,Sistema_nervioso_es.wikipedia.org_all-access_a...,2016-09-23,8066.0
349417,Creed_(film)_en.wikipedia.org_mobile-web_all-a...,2016-10-09,5770.0
719664,Vereinigtes_Königreich_de.wikipedia.org_mobile...,2016-11-09,1741.0
379354,Jennifer_Lopez_en.wikipedia.org_desktop_all-ag...,2016-10-12,4714.0
635144,Commons:Reusing_content_outside_Wikimedia_comm...,2016-11-02,3705.0
69311,"Уокер,_Пол_ru.wikipedia.org_all-access_all-agents",2016-09-15,2028.0


In [5]:
test['date'].min(), test['date'].max()

('2016-09-10', '2016-11-10')

In [6]:
train['date'].min(), train['date'].max()

('2016-01-01', '2016-08-31')

In [7]:
train.shape, test.shape

((2882860, 3), (732530, 3))

In [8]:
train.shape[0] / test.shape[0]

3.935483870967742

In [9]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [10]:
train['Month']     = train["date"].dt.month
train['Day']       = train["date"].dt.day
train['DayOfWeek'] = train["date"].dt.dayofweek

In [11]:
train_train, train_test = train[train['date'] < '2016-07-01'], train[train['date'] >= '2016-07-01']

In [12]:
train_train['date'].max()

Timestamp('2016-06-30 00:00:00')

In [13]:
temp = train_train[train_train['date'] > '2016-06-15']

In [14]:
pages = temp['Page'].unique()
y_pred = []

In [19]:
for page in pages:
    y_pred.append(temp[temp['Page'] == page]['Visits'].median())

In [20]:
i = 0
for page in pages:
    train_test.loc[train_test['Page'] == page, 'Visits_pred'] = y_pred[i]
    i += 1

In [27]:
def pandas_smape_error(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["Visits_pred"]) / (df["Visits"] + df["Visits_pred"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

In [28]:
def smape_fast_error(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred.iloc[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [29]:
smape_fast_error(train_test['Visits'], train_test['Visits_pred'])

41.016810453819595

In [30]:
pandas_smape_error(train_test)

41.01681045381974

In [31]:
pages

array(['Special:Search_zh.wikipedia.org_all-access_spider',
       'Wikipedia:首页_zh.wikipedia.org_all-access_spider',
       'Project:上传_zh.wikipedia.org_all-access_spider', ...,
       'Especial:Libro_es.wikipedia.org_all-access_spider',
       'Special:Search_es.wikipedia.org_all-access_spider',
       'Wikipedia:Portada_es.wikipedia.org_all-access_spider'],
      dtype=object)

In [32]:
i = 0
page_num = {pages[i]:i for i in range(len(pages))}

In [33]:
def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=2))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(months=2)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=4))].index

In [34]:
train_dates = ['2016-01-01', '2016-03-01', '2016-05-01']

In [35]:
test.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-09-10,640.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-09-10,5609.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-09-10,3121.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-09-10,362.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-09-10,533.0


In [36]:
train['page_num'] = train['Page'].map(page_num)

In [37]:
test['page_num'] = test['Page'].map(page_num)

In [38]:
X = train.drop(['Page', 'Visits','date'], axis = 1)
y = train['Visits']


X_test = test.drop(['Visits','date'], axis = 1)
y_test = test['Visits']

In [39]:
myCViterator = []
for i in train_dates:
    trainIndices, valIndices = create_validation(train, i)
    myCViterator.append( (trainIndices, valIndices) )

In [136]:
for x,y in myCViterator:
#     print(y)
    print (min(x), min(y))

0 708900
708900 1429615
1429615 2150330


In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [138]:
tree = RandomForestClassifier(random_state=17)

In [161]:
max_depth_val = range(13,16)
# max_features_values = range(4, 8)
params = {'max_depth': max_depth_val}

In [158]:
gcv = GridSearchCV(tree, params, n_jobs=-1, cv=myCViterator, scoring=smape_fast_error, verbose=1)
gcv.fit(X,y)


In [ ]:
# from sklearn.svm import SVR
# regressor = SVR()
from sklearn.pipeline import Pipeline
regressor = RandomForestClassifier(random_state=17)

pipeline_r = Pipeline([('regressor', re gressor)])
param_grid = [
    {'regressor__max_depth': max_depth_val}
#   {'regressor__C': [0.01, 0.1, 1, 10, 100, 1000], 'regressor__kernel': ['linear']},
#   {'regressor__C': [0.01, 0.1, 1, 10, 100, 1000], 'regressor__gamma': [0.001, 0.0001], 'regressor__kernel': ['rbf']},
 ]
grid_search = GridSearchCV(pipeline_r, param_grid=param_grid, cv=myCViterator, scoring=smape_fast_error)

%time grid_search.fit(X,y)

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [40]:
train = pd.read_csv('../data/train_french.csv')
test = pd.read_csv('../data/test_french.csv')

In [68]:
test.shape[0] / train.shape[0] 

0.37984

In [42]:
test.head()


,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,6029,5,OUI,500,Famille,agent; employe,Maison,1,7,C,...,0,164,41,0,0,1,NON,30,1,107.94
1,62128,2,NON,0,Famille,enseignant; formateur; chercheur,Appartement,1,1,D,...,1,85,100,0,0,1,NON,30,2,80.41
2,81065,4,NON,1000,Indivividu,homme ou femme au foyer,Sans information,2,99,~,...,0,94,100,0,0,1,NON,30,2,142.12
3,5023,3,NON,500,Indivividu,agent; employe,Maison,2,38,C,...,0,101,100,0,0,5,NON,30,2,76.55
4,62178,2,NON,500,Famille,enseignant; formateur; chercheur,Appartement,1,1,~,...,1,125,100,1,0,1,NON,0,2,150.45


In [72]:
from sklearn.model_selection import train_test_split

In [78]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [116]:
d = {'Famille': 0, 'Indivividu': 1}

In [113]:
X['var9'] = X['var9'].map(d)

In [117]:
X['sit_fam'] = X['sit_fam'].map(d)

In [118]:
X['nature_lri'].value_counts()

Appartement                        110302
Maison                              68488
Sans information                    19262
R�sidence de retraite                1145
Mobil home                            799
Local utilitaire distinct (LUD)         4
Name: nature_lri, dtype: int64

In [119]:
d = {'Appartement': 0,
'Maison': 1,
'Sans information': 2,
'R�sidence de retraite': 3,
'Mobil home': 4,
'Local utilitaire distinct (LUD)': 5}

In [120]:
X['nature_lri'] = X['nature_lri'].map(d)

In [122]:
X['veranda'].value_counts()

NON    194793
OUI      5207
Name: veranda, dtype: int64

In [123]:
d = {'NON': 0, 'OUI': 1}

In [125]:
X['veranda'] = X['veranda'].map(d)

In [127]:
X['niv_urbanisation'].value_counts()

A    42970
G    40950
B    39486
F    28154
D    19445
C    17506
~     6075
E     5414
Name: niv_urbanisation, dtype: int64

In [131]:
d = {'A': 0, 'G': 1, 'B': 2, 'F': 3, 'D': 4, 'C': 5, '~': 6, 'E': 7}

In [132]:
X['niv_urbanisation'] = X['niv_urbanisation'].map(d)

In [133]:
X.head()

,nb_pieces,veranda,biens_nomades,sit_fam,nature_lri,societe,domaine_activite,niv_urbanisation,age_societaire,objets_precieux,...,var2,var3,var4,var5,var6,var7,var8,var9,var10,var11
0,1,0,500,1,2,2,34,0,35,1,...,0,0,194,49,0,0,1,0,30,2
1,2,0,500,1,2,2,37,0,27,1,...,0,0,0,100,0,0,5,0,12,1
2,3,0,500,1,1,1,1,7,24,1,...,0,1,98,100,0,0,5,0,30,2
3,3,0,500,0,1,2,34,0,39,1,...,2,0,151,38,0,0,1,0,15,1
4,2,0,500,0,0,1,19,4,80,1,...,0,0,39,100,0,0,1,0,0,2


In [128]:
X = X.drop('profession', axis = 1)

In [81]:
X, y = train.drop(['code_postal', 'prime_tot_ttc'], axis=1), train['prime_tot_ttc']

In [86]:
np.sort(y)

array([ 32.06,  34.77,  34.77, ..., 324.42, 325.53, 469.67])

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=round(len(y)/3),
                                                    random_state=42)

In [105]:
tree = RandomForestClassifier(random_state=17)
max_depth_val = range(13,16)
# max_features_values = range(4, 8)
params = {'max_depth': max_depth_val}

In [158]:
y = train['prime_tot_ttc']

In [142]:
X.dropna(inplace=True)

In [159]:
y.dropna(inplace=True)

In [162]:
y = y.astype(int)

In [171]:
y.head()

0     67
1    120
2    100
3     58
4    132
Name: prime_tot_ttc, dtype: int64

In [176]:
%time
gcv = GridSearchCV(tree, params, verbose=1)
gcv.fit(X_train,y_train)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   39.0s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': range(13, 16)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=1)

In [177]:
gcv.best_score_

0.09080272700681752

In [179]:
y_pred = gcv.predict(X_test)

In [181]:
gcv.score(X_test, y_test)

0.09728951355243223

In [188]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [189]:
mape(y_test, y_pred)

19.545628956550047